# Cookbook -- OpenAI Recommendations Using Embeddings

This cookbook integrates lastmile-eval into [this example cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/Recommendation_using_embeddings.ipynb) from openai

## Setup
Set the following variables in your .env file:

OPENAI_API_KEY=\<your [openai api key](https://platform.openai.com/api-keys)\>

LASTMILE_API_TOKEN=\<your [lastmile api token](https://lastmileai.dev/settings?page=tokens)>

In [ ]:
# Install required packages for this cookbook
%pip install pandas
%pip install openai
%pip install python-dotenv
%pip install lastmile-eval

# For embeddings_utils
%pip install matplotlib
%pip install plotly
%pip install scipy
%pip install scikit-learn

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Start RAG Debugger Server

To start the server, open a terminal and run `rag-debug launch`

Alternatively, run the next cell and stop the cell after the server port is shown in the output. The server will continue running in the background and will be stopped when the kernel stops or restarts.

In [ ]:
%%bash 
rag-debug launch

# The server starts at the local port specified in the output. 
# Stop or restart the kernel to stop the RAG debugger server

In [ ]:
from lastmile_eval.rag.debugger.api.tracing import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="embedding_recommendations",
)

# Recommendation using embeddings and nearest neighbor search
Recommendations are widespread across the web.

- 'Bought that item? Try these similar items.'
- 'Enjoy that book? Try these similar titles.'
- 'Not the help page you were looking for? Try these similar pages.'
This notebook demonstrates how to use embeddings to find similar items to recommend. In particular, we use AG's corpus of news articles as our dataset.

Our model will answer the question: given an article, what other articles are most similar to it?

In [ ]:
import pandas as pd
import pickle

from utils.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    indices_of_nearest_neighbors_from_distances,
)

EMBEDDING_MODEL = "text-embedding-3-small"

## 2. Load data
Next, let's load the AG news data and see what it looks like.

In [ ]:
# load data (full dataset available at http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)
dataset_path = "data/AG_news_samples.csv"
df = pd.read_csv(dataset_path)

n_examples = 5
df.head(n_examples)

Let's take a look at those same examples, but not truncated by ellipses.

In [ ]:
# print the title, description, and label of each example
for idx, row in df.head(n_examples).iterrows():
    print("")
    print(f"Title: {row['title']}")
    print(f"Description: {row['description']}")
    print(f"Label: {row['label']}")

## 3. Build cache to save embeddings (DELETE cache file to re-create embeddings!)
Before getting embeddings for these articles, let's set up a cache to save the embeddings we generate. In general, it's a good idea to save your embeddings so you can re-use them later. If you don't save them, you'll pay again each time you compute them again.

The cache is a dictionary that maps tuples of `(text, model)` to an embedding, which is a list of floats. The cache is saved as a Python pickle file.

In [ ]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "data/recommendations_embeddings_cache.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
@tracer.start_as_current_span("embedding_from_string")
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

Let's check that it works by getting an embedding

In [ ]:
# as an example, take the first description from the dataset
example_string = df["description"].values[0]
print(f"\nExample string: {example_string}")

# print the first 10 dimensions of the embedding
example_embedding = embedding_from_string(example_string)
print(f"\nExample embedding: {example_embedding[:10]}...")

## 4. Recommend similar articles based on embeddings
To find similar articles, let's follow a three-step plan:

- Get the similarity embeddings of all the article descriptions
- Calculate the distance between a source title and all other articles
- Print out the other articles closest to the source title

In [ ]:
from lastmile_eval.rag.debugger.api import (
    QueryReceived,
)

@tracer.start_as_current_span("print_recommendations_from_strings")
def print_recommendations_from_strings(
    strings: list[str],
    index_of_source_string: int,
    k_nearest_neighbors: int = 1,
    model=EMBEDDING_MODEL,
) -> list[int]:
    # Register the relevant params from the function
    tracer.register_param("strings", strings)
    tracer.register_param("index_of_source_string", index_of_source_string)
    tracer.register_param("k_nearest_neighbors", k_nearest_neighbors)
    tracer.register_param("model", model)

    """Print out the k nearest neighbors of a given string."""
    # get embeddings for all strings
    embeddings = [embedding_from_string(string, model=model) for string in strings]

    # get the embedding of the source string
    query_embedding = embeddings[index_of_source_string]

    # get distances between the source embedding and other embeddings (function from utils.embeddings_utils.py)
    distances = distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine")
    
    # get indices of nearest neighbors (function from utils.utils.embeddings_utils.py)
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

    # print out source string
    query_string = strings[index_of_source_string]
    print(f"Source string: {query_string}")

    # Mark this query as QueryReceived event
    tracer.mark_rag_query_trace_event(QueryReceived(query=query_string))

    # print out its k nearest neighbors
    k_counter = 0
    for i in indices_of_nearest_neighbors:
        # skip any strings that are identical matches to the starting string
        if query_string == strings[i]:
            continue
        # stop after printing out k articles
        if k_counter >= k_nearest_neighbors:
            break
        k_counter += 1

        # print out the similar strings and their distances
        print(
            f"""
        --- Recommendation #{k_counter} (nearest neighbor {k_counter} of {k_nearest_neighbors}) ---
        String: {strings[i]}
        Distance: {distances[i]:0.3f}"""
        )

    return indices_of_nearest_neighbors

## 5. Example recommendations
Let's look for articles similar to first one, which was about Tony Blair.

In [ ]:
article_descriptions = df["description"].tolist()

tony_blair_articles = print_recommendations_from_strings(
    strings=article_descriptions,  # let's base similarity off of the article description
    index_of_source_string=0,  # articles similar to the first one about Tony Blair
    k_nearest_neighbors=5,  # 5 most similar articles
)

Pretty good! 4 of the 5 recommendations explicitly mention Tony Blair and the fifth is an article from London about climate change, topics that might be often associated with Tony Blair.

Let's see how our recommender does on the second example article about NVIDIA's new chipset with more security.

In [ ]:
chipset_security_articles = print_recommendations_from_strings(
    strings=article_descriptions,  # let's base similarity off of the article description
    index_of_source_string=1,  # let's look at articles similar to the second one about a more secure chipset
    k_nearest_neighbors=5,  # let's look at the 5 most similar articles
)

From the printed distances, you can see that the #1 recommendation is much closer than all the others (0.11 vs 0.14+). And the #1 recommendation looks very similar to the starting article - it's another article from PC World about increasing computer security. Pretty good!